In [27]:
# ac/utils.py
import numpy as np
import pandas as pd
from datetime import datetime
import importlib.util
from pathlib import Path
from arcticdb import Arctic, QueryBuilder

def initialize_db(db_path):
    global ac
    ac_local = Arctic(f'lmdb://{db_path}?map_size=5MB')
    
    if not "general" in ac_local.list_libraries():
        print("Creating library 'general' where *settings and *strategies will be stored")
        library = ac_local.get_library('general', create_if_missing=True)
        index_values = ['port', 
                        's3_db_management', # False for local
                        'aws_access_id', 'aws_access_key',
                        'bucket_name','region',
                        'start_tws','username','password']
        
        data = {'Value': ["7497", # default port
                        "False", # defaul local
                        "", "", # aws_access_id, key
                        "", "", # bucket_name, region
                        "False","","" # Start TWS Automatically default False
                        ]}
        df = pd.DataFrame(data, index=index_values)
        library.write(symbol="settings",data=df)
        ac = ac_local
        return ac
    
    else: # read local settings if settings table exists
        library = ac_local.get_library('general', create_if_missing=True)
        settings_df = library.read("settings").data
        
        # if S3 is set, change ac from local to s3
        if settings_df.loc["s3_db_management","Value"] == str(True):
            region = settings_df.loc["region","Value"]
            bucket_name = settings_df.loc["bucket_name","Value"]
            id = settings_df.loc["aws_access_id","Value"]
            key = settings_df.loc["aws_access_key","Value"]
        
            ac =Arctic(f's3://s3.{region}.amazonaws.com:{bucket_name}?region={region}&access={id}&secret={key}')

            # check if "settings" exists in s3 ac
            if not "general" in ac.list_libraries():
                lib = ac.get_library('general', create_if_missing=True)
                # copy settings from local ac
                lib.write("settings", settings_df)
            elif not "settings" in ac.get_library('general', create_if_missing=True).list_symbols():
                ac.get_library('general', create_if_missing=True).write("settings", settings_df)
        else:
            ac = ac_local
        return ac

In [16]:
ac = Arctic(f'lmdb://db?map_size=5MB')
lib = ac.get_library("general")

lib.delete("settings")

In [28]:
# ac = Arctic(f'lmdb://db?map_size=5MB')
ac = initialize_db("db")

In [19]:
ac.list_libraries()

['general', 'settings', 'spx500']

In [29]:
lib = ac.get_library('general')

In [30]:
lib.list_symbols()

['strategies', 'settings']

In [31]:
# lib.read('settings').data
lib.list_symbols()
lib.read('strategies').data

,name,filename,description,target_weight,min_weight,max_weight,params
SVIX,Short VIX Future,SVIX.py,If the Volatility Risk Premium is positive and...,0.05,0.02,0.08,"{'target_weight': 0.05, 'min_weight': 0.02, 'm..."
test1,test,test.py,,1,1,1,[Errno 2] No such file or directory: '/Users/j...


In [32]:
df = lib.read('strategies').data

In [33]:
df["active"] = True

In [34]:
lib.write('strategies',df)

VersionedItem(symbol='strategies', library='general', data=n/a, version=4, metadata=None, host='S3(endpoint=s3.eu-central-1.amazonaws.com, bucket=lowquant-arcticdb)')

In [35]:
df = lib.read('strategies').data

In [36]:
df

,name,filename,description,target_weight,min_weight,max_weight,params,active
SVIX,Short VIX Future,SVIX.py,If the Volatility Risk Premium is positive and...,0.05,0.02,0.08,"{'target_weight': 0.05, 'min_weight': 0.02, 'm...",True
test1,test,test.py,,1,1,1,[Errno 2] No such file or directory: '/Users/j...,True
